In [1]:
import pandas as pd
import time
import re
from datetime import datetime as dt, date
import json
import glob
import urllib

## Using scraped json (from Javascript)

Using https://github.com/nelsonic/github-scraper

In [2]:
path = 'data/01-in/github/2019-05-13/'
filepaths = [f for f in glob.glob(path + "*.json", recursive=True)]
df_init = pd.DataFrame(columns=['title', 'stars', 'link', 'progLang', 'desc', 'keywords'])

In [3]:
for f in filepaths:
    file = open(f, 'r')
    jso = json.loads(file.read())
    df = pd.DataFrame(jso['repos'])
    keywords = urllib.parse.parse_qs(jso['url'])['/search?q'][0]
    df = df.assign(keywords=keywords)
    df = df[~df.duplicated()]
    df_init = df_init.append(df, sort=False)

In [4]:
df_init.head()

,title,stars,link,progLang,desc,keywords
0,git2samus/xpost-bot,12,https://github.com//git2samus/xpost-bot,Python,Reddit bot to scan and repost submissions of i...,niche
1,bastodian/NichePy,7,https://github.com//bastodian/NichePy,Python,NichePy: a collection of Python scripts for es...,niche
2,dadams22/fynat,1,https://github.com//dadams22/fynat,Python,Find Your Niche At Tech,niche
3,ryanafolk/ambitus,4,https://github.com//ryanafolk/ambitus,Python,Ancestral niche reconstruction and associated ...,niche
4,NiteAx/opalescence,3,https://github.com//NiteAx/opalescence,Python,Niche-purpose discord moderation utility bot,niche


In [5]:
df_init.columns = ['name', 'stars', 'url', 'prog_lang', 'description', 'keywords']
df_init = df_init[['name', 'description', 'url', 'prog_lang', 'stars', 'keywords']]

In [6]:
df_init['name'] = df_init['name'].apply(lambda x: re.findall("([^/]+$)", x)[0])
df_init['url'] = df_init['url'].apply(lambda x: re.sub("https:\/\/github.com\/\/", "https://github.com/", x))

In [7]:
df_init.head()

,name,description,url,prog_lang,stars,keywords
0,xpost-bot,Reddit bot to scan and repost submissions of i...,https://github.com/git2samus/xpost-bot,Python,12,niche
1,NichePy,NichePy: a collection of Python scripts for es...,https://github.com/bastodian/NichePy,Python,7,niche
2,fynat,Find Your Niche At Tech,https://github.com/dadams22/fynat,Python,1,niche
3,ambitus,Ancestral niche reconstruction and associated ...,https://github.com/ryanafolk/ambitus,Python,4,niche
4,opalescence,Niche-purpose discord moderation utility bot,https://github.com/NiteAx/opalescence,Python,3,niche


In [8]:
df_init = df_init.reset_index()
uniques = df_init[~df_init['name'].duplicated()]
dups = df_init[df_init['name'].duplicated()]

print(df_init.shape, uniques.shape, dups.shape)

(726, 7) (585, 7) (141, 7)


In [9]:
dups.head()

,index,name,description,url,prog_lang,stars,keywords
60,0,fuzzy_habitat_modelling,Fish habitat model and visualization of habita...,https://github.com/jradinger/fuzzy_habitat_mod...,Python,,niche
114,0,ENMeval,❗️ This is a read-only mirror of the CRAN R pa...,https://github.com/cran/ENMeval,R,,niche
117,3,wallace,❗️ This is a read-only mirror of the CRAN R pa...,https://github.com/cran/wallace,R,,niche
132,2,nicheROVER,❗️ This is a read-only mirror of the CRAN R pa...,https://github.com/cran/nicheROVER,R,1,niche
137,7,MaxentVariableSelection,R package for Selecting the best set of releva...,https://github.com/alj1983/MaxentVariableSelec...,R,3,niche


In [10]:
for i, row in dups.iterrows():
    print(i, row['name'])
    
    if (not uniques.loc[uniques.name==row['name'], 'keywords'].str.contains(row["keywords"]).any()):
        uniques.at[uniques.name==row['name'], 'keywords'] = \
            ''.join(list(uniques[uniques.name==row['name']]['keywords'])) + f'; {row["keywords"]}'

60 fuzzy_habitat_modelling
114 ENMeval
117 wallace
132 nicheROVER
137 MaxentVariableSelection
181 CENFA
184 rKIN
192 phyloclim
253 SDM
269 SDM
270 sdm
271 SDM
272 SDM
273 SDM
274 sdm
276 sdm
277 sdm
278 SDM
279 sdm
298 SDMS
299 SDML
300 sdmpy
301 SDMS
304 SDML
305 sdml
307 sdms
310 sdmpy
312 sdms
313 SDMA
314 sdmreader
326 SDML-Final
327 sdm-project
330 sdml
334 SDMTools
338 enmSdm
350 Establishment_SDMs
351 ephedra_sdm
368 Carex.bipolar
379 biodiversity-sdm-lesson
409 GARPTools
417 SDMGroup
423 sdm
426 sdms
436 sdm
437 sdm
438 SDM
439 SDM
441 sdm
442 SDM
443 sdm
444 sdm
445 SDM
461 SDMPlay
462 SDMMetrics
463 SDMs
465 SDMR
466 SDMTools
467 sdms
470 SDMs
477 sdmvspecies
486 QTP_SDM
487 SDM_class
488 SDM_maxent
489 Mongolia_SDM
490 tree-sdm
493 NichePy


/home/euniceesh/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


499 SDMIT
501 sdm-python
503 SDM-GIS-tools
517 Regional_SDM
523 enmSdm
524 sdmbench
527 GARPTools
538 SDMGISR
539 ephedra_sdm
540 Niche-Shift-Circular-diagram
541 foram_sdm
548 SDMSeasonality
551 sdm
552 sdm
553 SDMTools
556 SDMtune
565 sdmvspecies
568 SDM-Class-Practicals
573 plant_gbif
580 maxlike
592 Climate-Change
593 sdmpredictors
594 wallace
595 ssdtools
596 Invasive-plant-abundance-SDM
599 Rcodes_MapNovelEnvironments_SDMs
601 GRaF
607 sdmpredictors
608 dismo
609 gp_sdm_paper
611 rSDM
613 SDMSelect
614 SDMGroup
615 biomod2
618 iSDM
619 species_distribution_models
620 species_distribution_models
621 Species-Distribution-Models
628 sdmexplain
629 SDM_course
633 SDM
637 SDMs
638 sdm-explorer
639 shinySDM
640 shinySDM
641 tree-sdm
642 sdms
647 FD-SDM
649 MountainBryophytesSDM
650 jSDM
654 PhyloSDM
655 nicheApport
657 sdms
660 SDM_maxent
661 Bamboo-SDM---CIB
662 sdm_snmb
663 SDMR
667 rf4sdm
671 SDMscripts
673 bioclim
679 sdmvspecies
684 gis_map_occupancy
690 lifeNiche
692 tree-range-v

In [11]:
if uniques.columns.str.contains('index').any():
    uniques = uniques.drop(['index'], axis=1)
uniques = uniques.assign(source="Github")
uniques = uniques[['name', 'description', 'url', 'keywords', 'prog_lang', 'source', 'stars']]
uniques.to_csv(f'data/02-out/pkgs-gith-{date.today()}.csv', index=False)

In [12]:
uniques

,name,description,url,keywords,prog_lang,source,stars
0,xpost-bot,Reddit bot to scan and repost submissions of i...,https://github.com/git2samus/xpost-bot,niche,Python,Github,12
1,NichePy,NichePy: a collection of Python scripts for es...,https://github.com/bastodian/NichePy,niche; species distribution,Python,Github,7
2,fynat,Find Your Niche At Tech,https://github.com/dadams22/fynat,niche,Python,Github,1
3,ambitus,Ancestral niche reconstruction and associated ...,https://github.com/ryanafolk/ambitus,niche,Python,Github,4
4,opalescence,Niche-purpose discord moderation utility bot,https://github.com/NiteAx/opalescence,niche,Python,Github,3
5,niche_vlaanderen,Python package to run the NICHE Vlaanderen model,https://github.com/inbo/niche_vlaanderen,niche,Python,Github,2
6,NicheLife,A map visualization tool to find your ideal ne...,https://github.com/priya-vijay/NicheLife,niche,Python,Github,1
7,nicheLang,"Updated May 22, 2018",https://github.com/mattHoyDev/nicheLang,niche,Python,Github,1
8,language-niche-research,Scientific repository for fiddling around with...,https://github.com/RichardLitt/language-niche-...,niche,Python,Github,2
9,NichePSO,"Updated Aug 22, 2018",https://github.com/bryaneduardo24/NichePSO,niche,Python,Github,


# With topic

In [15]:
file = open(f'data/01-in/github/github-topic-species-distribution-modelling-2019-05-14.json', 'r')
jso = json.loads(file.read())
df2 = pd.DataFrame(jso['repos'])

In [17]:
df2.columns = ['description', 'url', 'prog_lang', 'stars', 'name']

In [23]:
df2['name'] = df2['name'].apply(lambda x: re.findall("([^/]+$)", x)[0])
df2['keywords'] = 'topic (species distribution modelling)'

In [24]:
df2 = df2[['name', 'description', 'url', 'prog_lang', 'stars', 'keywords']]

In [29]:
df2.to_csv(f'data/02-out/pkgs-gith-{date.today()}-topics (species distribution modelling).csv', index=False)